In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

In [ ]:
def numneigh(i,x,h,npart):
  neigh = 0
  list = np.zeros(npart)
  for j in range(0,npart):
      if (abs(x[i]-x[j]) <= 2.0*h):
         list[neigh]=j
         neigh = neigh + 1
          
  return neigh, list        

In [ ]:
def kernel(q,h):
   W = 0.0
   if (q > 2.0):
     W = 0.0
   if ((q > 1.0) and (q < 2.0)):
     W = 0.25*(2-q)*(2-q)
   if (q <= 1.0):
     W = 1.0 - 1.5*q*q + 0.75*q*q*q

   W = 2.0*W/(3.0*h)

   return W

In [ ]:
def gradkernel(q,h):
   dWa = 0.0
   if (q > 2.0):
     dWa = 0.0
   if ((q > 1.0) and (q < 2.0)):
     dWa = -3.0*(2-q)*(2-q)/(4.0*h)
   if (q <= 1.0):
     dWa = -3.0*q/h + 9.0*q*q/(4.0*h)

   dWa = 2.0*dWa/(3.0*h)

   return dWa


In [ ]:
def density(i,x,smooth,nneigh,neighlist,mpart,npart):
  rho = 0.0
  for j in range(0,nneigh[i]):
       k = neighlist[i,j]
       q = abs(x[i] - x[k])/(0.5*(smooth[i]+smooth[k]))
       if (q <= 2.0):
          rho = rho + mpart*kernel(q,0.5*(smooth[i]+smooth[k]))

  return rho

In [ ]:
def accel(i,x,v,soundspeed,dens,Press,smooth,nneigh,neighlist,mpart,npart):
   dvadt = 0.0
   for j in range(0,nneigh[i]):
      k = neighlist[i,j]
      h = 0.5*(smooth[i]+smooth[k])
      q = abs(x[i] - x[k])/h
      if (q <= 2.0):
        if (i != k):
          temp = (Press[k]/(dens[k]*dens[k]) + Press[i]/(dens[i]*dens[i]))*(x[i]-x[k])*gradkernel(q,h)/abs(x[i]-x[k])
          dvadt = dvadt - mpart*temp

   return dvadt

In [ ]:
def energy(i,x,v,soundspeed,dens,Press,smooth,nneigh,neighlist,mpart,npart):
   dedt = 0.0

   return dedt

In [ ]:
npart = 1000
#mpart = 0.00125
mpart = 0.002
ho = 0.005
gamma = 1.0
#gamma = 1.66667

x = np.zeros(npart)
mass = np.zeros(npart)
v = np.zeros(npart)
dens = np.zeros(npart)
ee = np.zeros(npart)
Press = np.zeros(npart)
acc = np.zeros(npart)
soundspeed = np.zeros(npart)
edot = np.zeros(npart)
smooth = np.zeros(npart)
nneigh = np.zeros(npart,dtype=int)
neighlist = np.zeros((npart,npart),dtype=int)

In [ ]:
x[0] = -1.0
v[0] = 0.0
#ee[0] = 1.5
ee[0] = 0.5 
smooth[0] = ho
for i in range(1,npart):
  if (x[i-1] < 0.0):
#     x[i] = x[i-1] + 1.0/(0.8*npart)
#     ee[i] = 1.5
#     v[i] = 0.0
     x[i] = x[i-1] + 1.0/(0.7*npart)
     ee[i] = 1.0
     v[i] = 1.0 
  if (x[i-1] >= 0.0):
#     x[i] = x[i-1] + 1.0/(0.2*npart)
#     ee[i] = 1.0
#     v[i]=0.0
     x[i] = x[i-1] + 1.0/(0.3*npart)
     ee[i] = 1.0
     v[i] = 0.0 
  smooth[i] = ho

In [ ]:
for i in range(0,npart):
  nneigh[i] = npart
  for j in range(0,npart):
    neighlist[i,j] = j
  dens[i] = density(i,x,smooth,nneigh,neighlist,mpart,npart)

In [ ]:
dt = 0.0001
for j in range(0,2000):
  for i in range(0,npart):
     smooth[i] = ho/dens[i]**(1.0/3.0)
     nneigh[i],neighlist[i,:] = numneigh(i,x,smooth[i],npart)
     dens[i] = density(i,x,smooth,nneigh,neighlist,mpart,npart)
     Press[i] = (gamma - 1.0)*dens[i]*ee[i]
     soundspeed[i] = np.sqrt(gamma*Press[i]/dens[i])

  for i in range(0,npart):
     acc[i] = accel(i,x,v,soundspeed,dens,Press,smooth,nneigh,neighlist,mpart,npart)
     v[i] = v[i] + acc[i]*dt/2.0
     dens[i] = density(i,x+v[i]*dt/2.0,smooth,nneigh,neighlist,mpart,npart)
#     edot[i] = energy(i,x+v[i]*dt/2.0,v,soundspeed,dens,Press,smooth,nneigh,neighlist,mpart,npart)
     x[i] = x[i] + v[i]*dt
#     ee[i] = ee[i] + edot[i]*dt
     v[i] = v[i] + acc[i]*dt/2.0

#  print(j)  
  if (np.mod(j,50) == 0):
    plt.plot(x,v,ls='dashed',label='velocity')
    plt.plot(x,Press,ls='dotted',label='pressure')
    plt.plot(x,dens,label='density')

    plt.legend(loc='upper right')

    plt.xlim(-0.5,0.5)
    plt.ylim(-0.1,10.0)
    plt.show(block=False)
#    plt.pause(1)
    clear_output(wait=True)
#    plt.close()

plt.plot(x,v,ls='dashed',label='velocity')
plt.plot(x,Press,ls='dotted',label='pressure')
plt.plot(x,dens,label='density')

plt.legend(loc='upper right')

plt.xlim(-0.5,0.5)
plt.ylim(-0.1,10.0)
plt.show()

print('finished')